In [12]:
import abc
import dataclasses

In [13]:
@dataclasses.dataclass
class AbstractPoint(
    abc.ABC,
):

    @abc.abstractmethod
    def length(self):
        pass

In [14]:
pt = AbstractPoint()

TypeError: Can't instantiate abstract class AbstractPoint with abstract methods length

In [ ]:
@dataclasses.dataclass
class CartesianPoint(AbstractPoint):
    x: float
    y: float

    def length(self):
        return sqrt(self.x * self.x + self.y * self.y)

In [16]:
import kgpy.labeled
import kgpy.function
import kgpy.vectors

In [ ]:
kgpy.function.Polynomial(
    coefficients=kgpy.vectors.CartesianND({
        'x':
    })
)

In [19]:
x = kgpy.labeled.LinearSpace(0, 1, num=11, axis='x')
y = kgpy.labeled.LinearSpace(0, 1, num=11, axis='y')

In [22]:
intensity = x + y + 2 * x * y

In [31]:
a = kgpy.function.PolynomialArray(
    input=kgpy.vectors.Cartesian2D(x, y),
    output=kgpy.vectors.Cartesian2D(intensity, -intensity),
    degree=2,
)

In [32]:
a.coefficients

CartesianND(coordinates={'': Cartesian2D(x=Array(array=-5.329070518200751e-15, axes=[]), y=Array(array=5.329070518200751e-15, axes=[])), 'y': Cartesian2D(x=Array(array=0.9999999999999873, axes=[]), y=Array(array=-0.9999999999999873, axes=[])), 'x': Cartesian2D(x=Array(array=1.0, axes=[]), y=Array(array=-1.0, axes=[])), 'y,y': Cartesian2D(x=Array(array=5.423244716789147e-15, axes=[]), y=Array(array=-5.423244716789147e-15, axes=[])), 'x,y': Cartesian2D(x=Array(array=2.000000000000003, axes=[]), y=Array(array=-2.000000000000003, axes=[])), 'x,x': Cartesian2D(x=Array(array=1.4210854715202004e-14, axes=[]), y=Array(array=-1.4210854715202004e-14, axes=[]))})

In [33]:
import astropy.units as u
import kgpy.ctis.instruments
import kgpy.optics.aberrations

In [ ]:
instrument = kgpy.ctis.instruments.AberrationInstrument(
    aberration=kgpy.optics.aberrations.Aberration(
        distortion=kgpy.function.Polynomial(
            coefficients=kgpy.vectors.CartesianND({
                'x': kgpy.vectors.Cartesian2D(
                    x=kgpy.labeled.Array([1, 0] * u.pix / u.arcsec , axes=['channel']),
                    y=kgpy.labeled.Array([0, 1] * u.pix / u.arcsec , axes=['channel']),
                ),
                'y': kgpy.vectors.Cartesian2D(
                    x=kgpy.labeled.Array([0, 1] * u.pix / u.arcsec , axes=['channel']),
                    y=kgpy.labeled.Array([1, 0] * u.pix / u.arcsec , axes=['channel']),
                ),
                'wavelength': kgpy.vectors.Cartesian2D(
                    x=kgpy.labeled.Array([1, 1] * u.pix / u.arcsec , axes=['channel']),
                    y=kgpy.labeled.Array([0, 0] * u.pix / u.arcsec , axes=['channel']),
                ),
            })
        )
    )
)